In [2]:
from keras.applications.vgg16 import VGG16
from keras import models
import glob
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential
from tensorflow.python.layers.core import Dense
from nltk.metrics import scores
from sklearn import model_selection
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.layers import Flatten
import keras

In [9]:
def load_labels(myDir):
    labels = []
    fileList = glob.glob(myDir)
    for fname in fileList:
        fileName = os.path.basename(fname)
        curLabel = fileName.split("-")[0]
        labels.append(curLabel)
    return np.asarray(labels)

In [10]:
def load_data(myDir):
    fileList =  glob.glob(myDir)
    x = np.array([np.array(Image.open(fname)).flatten() for fname in fileList])
    x = x/255
    return x

In [11]:
myDir = "train/*.png"
labels = load_labels(myDir)
data = load_data(myDir)
Labels = tf.keras.utils.to_categorical(labels,10)

input_shape = (48,48,3)

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(data, Labels, test_size = 0.40, random_state = 42)

X_train_resized = []
for img in X_train:
    X_train_resized.append(np.resize(img, input_shape))
X_train_resized = np.array(X_train_resized)
print(X_train_resized.shape)

X_test_resized = []
for img in X_test:
    X_test_resized.append(np.resize(img, input_shape))
X_test_resized = np.array(X_test_resized)
print(X_test_resized.shape)


(4200, 48, 48, 3)
(2800, 48, 48, 3)


In [13]:
base_model = VGG16(weights='imagenet',include_top=False,input_shape=input_shape)
base_model.summary()

58900480/58889256 [==============================] - 12s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_____________________________________________________________

In [14]:
for layer in base_model.layers:
    layer.trainable = False
    print('Layers '+layer.name+' Frozen.')

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

epochs=10
batch_size=256

model.fit(X_train_resized, Y_train, batch_size=batch_size, validation_data=(X_test_resized, Y_test), epochs = epochs)
scores = model.evaluate(X_test_resized, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Layers input_1 Frozen.
Layers block1_conv1 Frozen.
Layers block1_conv2 Frozen.
Layers block1_pool Frozen.
Layers block2_conv1 Frozen.
Layers block2_conv2 Frozen.
Layers block2_pool Frozen.
Layers block3_conv1 Frozen.
Layers block3_conv2 Frozen.
Layers block3_conv3 Frozen.
Layers block3_pool Frozen.
Layers block4_conv1 Frozen.
Layers block4_conv2 Frozen.
Layers block4_conv3 Frozen.
Layers block4_pool Frozen.
Layers block5_conv1 Frozen.
Layers block5_conv2 Frozen.
Layers block5_conv3 Frozen.
Layers block5_pool Frozen.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312 